In [20]:
!pip install pandasql

In [21]:
import pandas as pd
from pandasql import sqldf

In [35]:
bankchurners = pd.read_csv('bankchurners.csv')
basic_client_info = pd.read_csv('basic_client_info.csv')
enriched_churn_data = pd.read_csv('enriched_churn_data.csv')


In [28]:
bankchurners.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   clientnum               10127 non-null  int64  
 1   attrition_flag          10127 non-null  object 
 2   dependent_count         10127 non-null  int64  
 3   card_category           10127 non-null  object 
 4   months_on_book          10127 non-null  int64  
 5   months_inactive_12_mon  10127 non-null  int64  
 6   contacts_count_12_mon   10127 non-null  int64  
 7   credit_limit            10127 non-null  float64
 8   avg_open_to_buy         10127 non-null  float64
 9   avg_utilization_ratio   10127 non-null  float64
dtypes: float64(3), int64(5), object(2)
memory usage: 791.3+ KB


In [32]:
query = """
SELECT 
    *
FROM 
    bankchurners
        """
sqldf(query)

TypeError: __init__() got multiple values for argument 'schema'

#### Question 1
 What’s the total churn rate among all clients?

In [36]:
query = """
With total_count as 
     (SELECT count(*)
     from bankchurners)

SELECT 
      CONCAT(ROUND(100.0*(SELECT count(*)
      from bankchurners
      where attrition_flag = 'Attrited Customer')/
                               (select * from total_count), 2), '%')
 """
sqldf(query)

TypeError: __init__() got multiple values for argument 'schema'

#### Question 2: 
2. For each attrition flag group (churn group), how many clients are there and what’s their average card utilization? Order by the avg. utilization in a descending order.

In [ ]:
query = """
SELECT attrition_flag, count(*), avg(avg_utilization_ratio)
FROM bankchurners
GROUP BY 1
Order by 3
        """
sqldf(query)

#### Question 3:
    How does the Card Utilization Ratio affect churn? Show results using 10% interval of utilization groups. 
    Could you notice groups which are more likely to churn? What might be the reasons for churn?
    

In [38]:
 
query = """
SELECT
    ROUND(CAST(FLOOR(avg_utilization_ratio/0.1)*0.1
               AS NUMERIC), 2) AS util_10_per,
    ROUND(AVG(CASE WHEN attrition_flag = 'Attrited Customer'
                   THEN 1.0 ELSE 0.0 END), 3) AS churn_rate,
    COUNT(*) AS clients      
FROM bankchurners
GROUP BY 1
ORDER BY 1
        """
sqldf(query)

IndentationError: unexpected indent (<ipython-input-38-b17b57d35656>, line 2)

#### Question 4:
    How does credit card type (category) affect churn? [increases/ decreases / stable / unable to determine].
    Sort the results using the following order: [Blue, Silver, Gold, Platinum].

In [ ]:
query = """
SELECT
     CASE WHEN card_category = 'Blue' THEN 1
         WHEN card_category = 'Silver' THEN 2
         WHEN card_category = 'Gold' THEN 3
         WHEN card_category = 'Platinum' THEN 4
         ELSE 0 END AS card_category_num,
         card_category,
    ROUND(AVG(CASE WHEN attrition_flag = 'Attrited Customer'
                   THEN 1.0 ELSE 0.0 END), 3) AS churn_rate,
    COUNT(*) AS clients      
FROM bankchurners
GROUP BY 1,2
ORDER BY 1

        """
sqldf(query)

#### Question 5: 
    Do the number of dependents affect the average credit limit? [a. Yes, increases; b. Yes, decreases; c. No affect; d. Unable  to tell]. Does the result make sense?


In [ ]:
query = """
SELECT
    dependent_count,
    AVG(credit_limit) AS avg_credit_limit,
    COUNT(*) AS clients      
FROM bankchurners
GROUP BY 1
ORDER BY 1
        """
sqldf(query)

#### Question 6:
    Do tenured customers (months_on_book >= 36) tend to churn more than new customers? (also add the number of clients in each group).

In [ ]:
# entails definning a case statement for tenued and non tenured customers
# claculate the churn percent
query = """

SELECT
    CASE WHEN months_on_book >= 36 THEN 1 ELSE 0 END AS tenured,
    AVG(CASE WHEN attrition_flag = 'Attrited Customer' THEN 1 ELSE 0 END) AS churn_rate,
    COUNT(*) AS clients      
FROM bankchurners
GROUP BY 1
ORDER BY 1
        """
sqldf(query)

In [ ]:
#### Question 7:
Do highly educated persons have higher income than basic educated persons?


##### Note: this question in particular can be answered in a wide number of ways given that income is an ordinal variable and not numerical. Both income and education have a high number of categories, so one option is to reduce the number of categories for income and simply look at the percent of people with greater than $60K income. This solution “pivots” the data to make visual comparison easier.

a. Result type: Aggregated table

b. Source data: basic_client_info

c. Process:
i. Write a subquery to get the number of people at each education level that have an income of less/more than $60k
1. Group by education level
2. Conditional agg number of clients per income (case + count) and total clients (count)

ii. From that subquery, calculate the percent in each reduced income group
iii. Create a sorting column for education level
iv. Order by the sorting column


In [ ]:
WITH income_subtotals AS (
   SELECT education_level,
       COUNT(clientnum) AS total_clients,
       COUNT(CASE WHEN income_category IN 
                   ('Less than 40K', '$40K - $60K')
                   THEN clientnum ELSE NULL END) 
            AS clients_0_60k,
       COUNT(CASE WHEN income_category IN 
                   ('$60K - $80K', '$80K - $120K', '$120K +')
                   THEN clientnum ELSE NULL END) 
            AS clients_60k_plus,
        COUNT(CASE WHEN income_category = 'Unknown'
                   THEN clientnum ELSE NULL END) 
            AS clients_unknown
    FROM basic_client_info
    GROUP BY 1
)


SELECT
    CASE WHEN education_level = 'Uneducated' THEN 1
         WHEN education_level = 'High School' THEN 2
         WHEN education_level = 'College' THEN 3
         WHEN education_level = 'Graduate' THEN 4
         WHEN education_level = 'Post-Graduate' THEN 5
         WHEN education_level = 'Doctorate' THEN 6
         WHEN education_level = 'Unknown' THEN 7
         ELSE -1 END AS education_num,
    education_level,
    total_clients,
    ROUND(1.0*clients_0_60k/total_clients, 3) 
        AS percent_0_60k,
    ROUND(1.0*clients_60k_plus/total_clients, 3) 
        AS percent_60k_plus,
    ROUND(1.0*clients_unknown/total_clients, 3) 
        AS percent_unknown
FROM income_subtotals
ORDER BY 1